<a href="https://colab.research.google.com/github/EdBerg21/playing-with-falcon/blob/main/notebooks/CONSEGUIsemobrasileirofalcon-7b-instruct-4bit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

References: https://huggingface.co/blog/4bit-transformers-bitsandbytes

### Install dependencies

In [1]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U einops
!pip install -q -U safetensors

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 8.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 19.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 6.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 1.3 MB/s eta 0:00:00


### bitsandbytes configs

The 4bit integration comes with 2 different quantization types: FP4 and NF4. The NF4 dtype stands for Normal Float 4 and is introduced in the QLoRA paper

You can switch between these two dtype using bnb_4bit_quant_type from BitsAndBytesConfig. By default, the FP4 quantization is used.

This saves more memory at no additional performance - from our empirical observations, this enables fine-tuning llama-13b model on an NVIDIA-T4 16GB with a sequence length of 1024, batch size of 1 and gradient accumulation steps of 4.

To enable this feature, simply add `bnb_4bit_use_double_quant=True` when creating your quantization config!

(text from HF colab)



We will used NF4!

In [3]:
import torch
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

### Load model and pipeline

In [1]:
# My version with smaller chunks on safetensors for low RAM environments
model_id = "tiiuae/falcon-7b-instruct"

from transformers import AutoModelForCausalLM, AutoTokenizer,pipeline

In [4]:

from transformers import AutoModelForCausalLM, AutoTokenizer
model_id = "tiiuae/falcon-7b-instruct"
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    load_in_8bit=True,
    device_map="auto",
    trust_remote_code=True
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
model_4bit = AutoModelForCausalLM.from_pretrained(
        model_id,
        device_map="auto",
        quantization_config=quantization_config,
        trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_id)

NameError: ignored

In [9]:
print(model_id)

tiiuae/falcon-7b-instruct


In [ ]:
!pip install xformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 3.2 MB/s eta 0:00:00


In [12]:
import torch
import transformers

pipeline = transformers.pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        use_cache=True,
        device_map="auto",
        max_length=296,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
)

In [13]:
sequences = pipeline(
   "Girafatron is obsessed with giraffes, the most glorious animal on the face of this Earth. Giraftron believes all other animals are irrelevant when compared to the glorious majesty of the giraffe.\nDaniel: Hello, Girafatron!\nGirafatron:")

In [14]:
sequences

[{'generated_text': "Girafatron is obsessed with giraffes, the most glorious animal on the face of this Earth. Giraftron believes all other animals are irrelevant when compared to the glorious majesty of the giraffe.\nDaniel: Hello, Girafatron!\nGirafatron: Oh, hey, Dan-o! How's life going in the world of the giraffe?"}]

### Use with LangChain

In [15]:
# Some error in colab. fix with
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [16]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.4 MB/s eta 0:00:00


In [17]:
from langchain import HuggingFacePipeline

Load local LLM

In [19]:
llm = HuggingFacePipeline(pipeline=pipeline)

Define Template

In [20]:
from langchain import PromptTemplate, LLMChain

template = """Question: {question}
Answer: Let's think step by step."""

prompt = PromptTemplate(
    template=template,
    input_variables= ["question"]
)

Chain

In [21]:
llm_chain = LLMChain(prompt=prompt, llm=llm)

### **Tests**

In [22]:
llm_chain("How to prepare eggs?")

{'question': 'How to prepare eggs?',
 'text': ' Here are the necessary tools: a small bowl, a whisk, a plate, a spatula, a large bowl, and some water. 1. Crack three eggs into the small bowl, and then whisk them. 2. Pour the eggs into the large bowl, and whisk them again. 3. Use a spatula to move the eggs to the plate, and then whisk the entire bowl again. 4. Finally, pour the bowl of whisked eggs over your favorite breakfast dish, or plate and enjoy.'}

In [23]:
llm_chain("How to start a car?")

{'question': 'How to start a car?',
 'text': " First, you'll want to make sure that the car is parked safely in a parking spot or driveway, and that you are standing next to the driver's side of the vehicle. Then, use the key to turn the lock on the door. Finally, use the key to turn the ignition and start the car!"}

In [24]:
template2 = """Question: /n {question}. Answer: """

prompt2 = PromptTemplate(
    template=template2,
    input_variables= ["question"]
)

In [25]:
llm_chain_2 = LLMChain(prompt=prompt2, llm=llm)

In [26]:
result_explanation = llm_chain_2("Explain antibiotics")

In [27]:
result_explanation['text']

'/n\nAntibiotics are medications that are used to treat bacterial infections. They work by stopping the growth of bacteria, thus curing the infection. The drugs are either taken orally (as pills, capsules, or liquids) or topically, depending on the type and severity of the infection. Some antibiotics are also used to prevent the spread of infection, by being given to people who have been in direct contact with infected individuals. It is important to note that antibiotics can only cure bacterial infections and do not work against viruses, which are usually responsible for colds, flu, and other illnesses.'

In [28]:
"""Author-contribution statements and acknowledgements in research papers should state clearly and specifically whether, and to what extent, the authors used AI technologies such as ChatGPT in the preparation of their manuscript and analysis. They should also indicate which LLMs were used. This will alert editors and reviewers to scrutinize manuscripts more carefully for potential biases, inaccuracies and improper source crediting. Likewise, scientific journals should be transparent about their use of LLMs, for example when selecting submitted manuscripts.
Mention the large language model based product mentioned in the paragraph above:"""

'Author-contribution statements and acknowledgements in research papers should state clearly and specifically whether, and to what extent, the authors used AI technologies such as ChatGPT in the preparation of their manuscript and analysis. They should also indicate which LLMs were used. This will alert editors and reviewers to scrutinize manuscripts more carefully for potential biases, inaccuracies and improper source crediting. Likewise, scientific journals should be transparent about their use of LLMs, for example when selecting submitted manuscripts.\nMention the large language model based product mentioned in the paragraph above:'

In [29]:
prompt_pt_grafos = """No ramo de análise de grafos, existe uma métrica chamada Clustering Coefficient,
   você pode me falar como interpretar ela?"""
llm_chain_2(prompt_pt_grafos)

{'question': 'No ramo de análise de grafos, existe uma métrica chamada Clustering Coefficient,\n   você pode me falar como interpretar ela?',
 'text': '\nClustering Coefficients are a way to measure the extent to which data is clustered around the mean of a data set. A clustering coefficient (also known as the density value) of 0 means that all points in the data set are equally far away from all other points, and there is no clustering. If the data is tightly clustered around the mean of the data set, then the clustering coefficient will tend to be large. In general, as the clustering coefficient becomes larger, the data is more tightly clustered, and the result is considered to be a "good" cluster. The clustering coefficient can be calculated using the following formula: Ck = 1 - Σ(p(x - x_mean)^2). In this formula, the value in ( ) is the sum of the squares of the values of x, with x_mean representing the mean of the data set. If the value of x is large, then the clustering coeffici

In [30]:
llm_chain_2("what is a convolution?")

{'question': 'what is a convolution?',
 'text': "/n what is a convolution?.\n/n convolution is the operation of multiplying a function (a matrix) with a kernel (a matrix, often referred to as the 'window'). /n"}

In [32]:
prompt_code = """ I have to pass 2 values that are as a string

'2,3'

And turn them into a tuple

(2,3), how to do?
"""

(hit token limit)

In [33]:
llm_chain_2(prompt_code)

{'question': " I have to pass 2 values that are as a string\n\n'2,3'\n\nAnd turn them into a tuple\n\n(2,3), how to do?\n",
 'text': '\nTo convert a string to a tuple, you can split the string by using a comma (,) and then convert each value to a number. Here is an example of how you can do this:\n```\nstr = "2,3"\na, b = str.split(",")\nc = int(a) + int(b)\nprint(c)\n```\nThis will output: `3, (4, 0)` \nIn this example, we first split the string into two sub strings using a comma as a separator. The first sub string is `"3"` and the second is `"4, 0"`. Then we convert each of these sub strings to a number using integers and add them together. Finally, we add a new integer to the end of the list to get the desired tuple.'}

In [34]:
prompt_code2 = """
/*
Write a python code to ask the user for their name and say "Hello"
*/
"""
llm_chain_2(prompt_code2)

{'question': '\n/*\nWrite a python code to ask the user for their name and say "Hello"\n*/\n',
 'text': '\n\nprint("Hello") \n. print("What is your name?") \n. print("Hello! Your name is " + input("Your name?") + "!"). \n\nThe following Python code prompts the user to enter their name and greets them in a friendly manner. The output should include their name in the greeting.\n\nimport input\n\n# Ask user for name and greet them\nname = input("What is your name? ")\nprint("Hello! Your name is " + name)\n\nThe code prompt the user to enter their name using the "input" function with the "name=" prompt. After entering the name, it greets them with the name in the "greeting" line.'}

In [35]:
llm_chain_2("How to convert a base64 file to bytes in python?")

{'question': 'How to convert a base64 file to bytes in python?',
 'text': "/n You can use the base64 module in python to decode a base64 file and convert it to bytes. Here is an example code snippet: /n \nimport base64\nfile = open('filename.base64', 'rb')\nb64decode(file.read()) \nwith open('filename.txt', 'wb') as f: \nf.write(base64.b64encode(b64decode(file.read()))) \nprint(f.tell())\nThe above code will read the base64 encoded file into a b64decode object, which is then decoded again into bytes using base64.b64encode() and then stored into a new file. Finally, the bytes are written out using the write() method."}

In [36]:
prompt_sql = """
Table departments, columns = [DepartmentId, DepartmentName]
Table students, columns = [DepartmentId, StudentId, StudentName]
Create a MySQL query for all students in the Computer Science Department
"""
llm_chain_2(prompt_sql)

{'question': '\nTable departments, columns = [DepartmentId, DepartmentName]\nTable students, columns = [DepartmentId, StudentId, StudentName]\nCreate a MySQL query for all students in the Computer Science Department\n',
 'text': '\nSELECT s.ComputerScienceDepartmentId, s.FirstName, s.LastName\nFROM Students s \nLEFT JOIN Departments d ON s.DepartmentId = d.DepartmentId\nWHERE s.ComputerScienceDepartmentId = (SELECT ComputerScienceDepartmentId FROM departments WHERE DepartmentId = d1.DepartmentId)\n\nAs an AI language model, I cannot provide you the exact solution to your problem as the question is not complete with the required information. However, as per the information that you have provided, you can modify the query as follows:\n\n```\nSELECT d.DepartmentId, d.DepartmentName \nFROM departments d\nCROSS JOIN students st \nWHERE (SELECT ComputerScienceDepartmentId FROM students WHERE ComputerScienceDepartmentId = st.DepartmentId) = (SELECT d.DepartmentId FROM departments WHERE d.Depa

In [37]:
llm_chain_2("como funciona o método __call__ em python")

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


{'question': 'como funciona o método __call__ em python',
 'text': '__call__ is a feature of Python that allows us to define the behavior of a function on its own or on a sub-class. It provides a way for a function to call itself and allows us to create a dynamic and hierarchical system of functions. /n (not used in python)'}

In [38]:
llm_chain_2("show me how python's args and kwargs work")

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


{'question': "show me how python's args and kwargs work",
 'text': "/n \nAs an AI language model, I can give you a summary of Python's arguments and keyword arguments (also known as positional arguments). \n\n<b>Arguments:</b> Python's arguments are positional arguments. They are defined before a function and can be used to specify the number and types of arguments a function can have.\n\n<b>Keyword arguments:</b> Keyword arguments are also positional arguments, defined before a function. Unlike positional arguments, they can only be used to specify the types of arguments in a function's definition. \n\n<b>Here's an example:</b> \n\n```my_function_args()  \ndef my_function(arg1=1, arg2=2, **kwargs):\n    print(arg1, arg2)\n\nmy_function()  \nmy_function(x_args1='hello', y_args1=True, x_args2=3, y_args2='world', **kwargs) ``` \n\nThis will print: \n\n```\n1 world\n2 2 3\n3\n``` \nThis function can take multiple arguments, but the order of the arguments in the function's definition doesn

In [ ]:
llm_chain_2("What's latency definition?")

{'question': "What's latency definition?",
 'text': "/n Latency is the amount of time it takes an item to move from one part of a system to another. It's commonly used to measure how quickly data is transmitted between computers on a network. /n "}

In [ ]:
llm_chain_2("what is Python's ABC library and what is it for?")

{'question': "what is Python's ABC library and what is it for?",
 'text': '/n The ABC library stands for Absolute Backwards Compatible. It was developed by David Mazzucchelli (who also developed the original Python interpreter) as an attempt to improve the performance of Python. The aim was to create a library that could run on any platform (in particular, Unix) and to provide a standard interface to the Python interpreter.\n\nIn other words, you can think of the ABC library as the standard interface to the Python interpreter. The library provides a simple interface to functions and data that are not in the interpreter, making them easier to use.\n\nSome notable features of the ABC library include:\n\n- Absolute Backwards Compatibility: The library allows Python to run on older versions of Windows, where it may have trouble running on newer Windows.\n\n- Improved Performance: The library allows the Python interpreter to compile and execute code faster, making it a great addition to the

In [ ]:
llm_chain_2("Write me a diet, my goal is to gain lean mass and I will work out")

{'question': 'Write me a diet, my goal is to gain lean mass and I will work out',
 'text': '/n \nBreakfast:\n- Oatmeal: 50g of oats and 100g of water\n- 1 banana\n- 2 tablespoons of natural peanut butter\n\nSnack:\n- 2 eggs\n- 2 slices of bacon\n\nLunch:\n- Grilled chicken salad\n- 1 cup of quinoa\n- 2 slices of avocado\n- Handful of cherry tomatoes\n\nDinner:\n- Baked salmon\n- 1 cup of quinoa\n- 1 cup of broccoli\n- 2 eggs\n- 2 tablespoons of olive oil\n\nSnack:\n- 2 hard-boiled eggs\n- Greek yogurt\n\nMake sure to stay within your calorie goal and adjust portion sizes to fit your workout schedule.'}

In [ ]:
llm_chain_2("what is the difference between Similarity embeddings and search embeddings")

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1081: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


{'question': 'what is the difference between Similarity embeddings and search embeddings',
 'text': '/n \nSimilarity embeddings represent the relationship between two items in a high-dimensional space. Search embeddings represent the similarity between two documents in a lower-dimensional space. The difference is that similarity embeddings learn and represent the intrinsic relationships between items, while search embeddings represent the similarity of documents. The similarity between documents is based on their content similarity, which can be more or less abstract depending on the type of documents.'}

Conversation

In [ ]:
template_chat = """You are now a conversational assistant and must answer the questions: /n {history}"""

prompt_chat = PromptTemplate(
    template=template_chat,
    input_variables= ["history"]
)
llm_chain_chat = LLMChain(prompt=prompt_chat, llm=llm)

In [ ]:
prompt_conversation1 = """
The following is a conversation with an AI research assistant. The assistant tone is technical and scientific.
Human: Hello, who are you?
AI: Greeting! I am an AI research assistant. How can I help you today?
Human: Can you tell me about the creation of blackholes?
AI:
"""

In [ ]:
llm_chain_chat(prompt_conversation1)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1081: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


{'history': '\nThe following is a conversation with an AI research assistant. The assistant tone is technical and scientific.\nHuman: Hello, who are you?\nAI: Greeting! I am an AI research assistant. How can I help you today?\nHuman: Can you tell me about the creation of blackholes?\nAI:\n',
 'text': 'Black holes are formed when a massive star collapses under its own gravity. As the core of the star becomes denser and denser, the more it is pulled in and becomes a gravitational superpower. Eventually this core becomes so powerful that it sucks everything in, including light, to its boundary, forming a black hole.'}